In [1]:
from pyomo.environ import Param,TransformationFactory
from pyomo.network import Arc
from idaes.core.util.scaling import constraint_scaling_transform
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Product, Feed, Mixer, Separator
from idaes.models.unit_models.separator import SplittingType
from idaes.core import UnitModelCostingBlock
from watertap.unit_models.pressure_changer import Pump, EnergyRecoveryDevice
from watertap.costing import WaterTAPCosting
from watertap.unit_models.reverse_osmosis_0D import (ReverseOsmosis0D, ConcentrationPolarizationType, 
                                                     MassTransferCoefficient, PressureChangeType)
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
import watertap.property_models.seawater_prop_pack as properties
from pyomo.environ import ConcreteModel, Var, Reals, Objective, Constraint, value, units, Expression
from idaes.core.solvers import get_solver
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from watertap.costing.zero_order_costing import ZeroOrderCosting
from watertap.core.wt_database import Database
from idaes.models.unit_models.translator import Translator
import csv

# setup model
m = ConcreteModel()
# m.db = Database()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = properties.SeawaterParameterBlock()
m.fs.costing = WaterTAPCosting()  # costing model

# create unit models
m.fs.feed = Feed(property_package=m.fs.properties)


ppichem2013 = 187.4 #jan2013, for Voutchkov, WPU06790961
ppichem2023 = 217.771 #july2023, WPU06790961

cepci1995 = 381.1
cepci2012 = 584.6
cepci2017 = 567.5
cepci2018 = 603.1
cepci2023 = 793.3 #September 2023
eleccost = 8.09/100 #$/kWh, industrial, October 2023

#indices for operating labor from stlouisfed, https://fred.stlouisfed.org/series/ECIWAG

eciq22022 = 154.0
eci2023 = (159.4+161.1+162.7+164.4)/4

eciratio = eci2023 / eciq22022

#eciratio = 1
#mixer for removal of solids
m.fs.mixer = Mixer(property_package=m.fs.properties, inlet_list=["feed"])

#separator for removal of solids
m.fs.separator = Separator(property_package=m.fs.properties, outlet_list=["solids","feed1"], split_basis=SplittingType.componentFlow)
# m.fs.RO = ReverseOsmosis0D(
#         property_package=m.fs.properties,
#         has_pressure_change=True,
#         pressure_change_type=PressureChangeType.calculated,
#         mass_transfer_coefficient=MassTransferCoefficient.calculated,
#         concentration_polarization_type=ConcentrationPolarizationType.calculated)
#initial pump
m.fs.pump = Pump(property_package=m.fs.properties)

#brackish water RO
m.fs.RO = ReverseOsmosis0D(property_package=m.fs.properties,
                             has_pressure_change=False,
                             concentration_polarization_type=ConcentrationPolarizationType.none,
                             mass_transfer_coefficient=MassTransferCoefficient.none)

m.fs.separator2 = Separator(property_package=m.fs.properties, outlet_list=["solids","feed1"], split_basis=SplittingType.componentFlow)
#seawater RO
m.fs.RO2 = ReverseOsmosis0D(property_package=m.fs.properties,
                             has_pressure_change=False,
                             concentration_polarization_type=ConcentrationPolarizationType.none,
                             mass_transfer_coefficient=MassTransferCoefficient.none)

# m.fs.RO2 = ReverseOsmosis0D(
#         property_package=m.fs.properties,
#         has_pressure_change=True,
#         pressure_change_type=PressureChangeType.calculated,
#         mass_transfer_coefficient=MassTransferCoefficient.calculated,
#         concentration_polarization_type=ConcentrationPolarizationType.calculated)


m.fs.product = Product(property_package=m.fs.properties)
m.fs.product2 = Product(property_package=m.fs.properties)
m.fs.brine = Product(property_package=m.fs.properties)
m.fs.solids = Product(property_package=m.fs.properties)
m.fs.solids2 = Product(property_package=m.fs.properties)
m.fs.RO2inlet = Product(property_package=m.fs.properties)

# connect unit models
m.fs.s1 = Arc(source=m.fs.feed.outlet, destination=m.fs.mixer.feed)
m.fs.s2 = Arc(source=m.fs.mixer.outlet, destination=m.fs.separator.inlet)
m.fs.s_s = Arc(source=m.fs.separator.solids, destination=m.fs.solids.inlet)
m.fs.s3 = Arc(source=m.fs.separator.feed1, destination=m.fs.pump.inlet)
m.fs.s4 = Arc(source=m.fs.pump.outlet, destination=m.fs.RO.inlet)
m.fs.s5 = Arc(source=m.fs.RO.permeate, destination=m.fs.product.inlet)
m.fs.s6 = Arc(source=m.fs.RO.retentate, destination=m.fs.separator2.inlet)
m.fs.s_s2 = Arc(source=m.fs.separator2.solids, destination=m.fs.solids2.inlet)
m.fs.s7 = Arc(source=m.fs.separator2.feed1, destination=m.fs.RO2.inlet)
m.fs.s7_1 = Arc(source=m.fs.separator2.feed1, destination=m.fs.RO2inlet.inlet)
m.fs.s8 = Arc(source=m.fs.RO2.permeate, destination=m.fs.product2.inlet)
m.fs.s9 = Arc(source=m.fs.RO2.retentate, destination=m.fs.brine.inlet)

TransformationFactory("network.expand_arcs").apply_to(m)

# unit equipment capital and operating costs
m.fs.mixer.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
m.fs.pump.work_mechanical[0].setlb(0)
m.fs.pump.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
m.fs.RO.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing)
m.fs.RO2.costing = UnitModelCostingBlock(flowsheet_costing_block=m.fs.costing, costing_method_arguments = {'ro_type': 'high_pressure'})
m.fs.costing.antiscalant_cost = Var(initialize=0.1,bounds=(0,None),doc="antiscalant cost",units=units.USD_2018 / units.L)
m.fs.costing.register_flow_type("antiscalant", m.fs.costing.antiscalant_cost)
m.fs.costing.cost_flow(m.fs.feed.properties[0].flow_vol_phase["Liq"], "antiscalant")
# system costing - total investment and operating costs
m.fs.costing.cost_process()
m.fs.costing.add_annual_water_production((m.fs.product.properties[0].flow_vol+m.fs.product2.properties[0].flow_vol))
m.fs.costing.add_specific_energy_consumption((m.fs.product.properties[0].flow_vol+m.fs.product2.properties[0].flow_vol))
m.fs.costing.add_LCOW((m.fs.product.properties[0].flow_vol+m.fs.product2.properties[0].flow_vol))

# scaling
# set default property values
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))
# set unit model values
set_scaling_factor(m.fs.pump.control_volume.work, 1e-3)
set_scaling_factor(m.fs.RO.area, 1e-2)
set_scaling_factor(m.fs.RO2.area, 1e-2)

# touch properties used in specifying the model
m.fs.feed.properties[0].flow_vol_phase["Liq"]
m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"]
m.fs.brine.properties[0].flow_vol_phase["Liq"]
m.fs.brine.properties[0].mass_frac_phase_comp["Liq", "TDS"]
m.fs.solids.properties[0].flow_vol_phase["Liq"]
m.fs.solids.properties[0].mass_frac_phase_comp["Liq", "TDS"]
m.fs.solids2.properties[0].flow_vol_phase["Liq"]
m.fs.solids2.properties[0].mass_frac_phase_comp["Liq", "TDS"]
m.fs.RO2inlet.properties[0].flow_vol_phase["Liq"]
m.fs.RO2inlet.properties[0].mass_frac_phase_comp["Liq", "TDS"]

# calculate and propagate scaling factors
calculate_scaling_factors(m)

print("DOF = ", degrees_of_freedom(m))

# feed, 4 degrees of freedom
m.fs.feed.properties[0].flow_vol_phase["Liq"].fix(0.003899*1)                # volumetric flow rate (m3/s), equal to 61.8 gpm (base)
m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(0.002829)  # TDS mass fraction (-), base 2829
#m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(0.4857)
m.fs.feed.properties[0].pressure.fix(101325)                           # pressure (Pa)
m.fs.feed.properties[0].temperature.fix(273.15 + 25)                   # temperature (K)

# separator, 1 degree of freedom
m.fs.separator.split_fraction[0, "solids", "TDS"].fix(1e-5)
m.fs.separator.split_fraction[0, "solids", "H2O"].fix(1e-5)
# high pressure pump, 2 degrees of freedom
m.fs.pump.efficiency_pump.fix(0.80)                                    # pump efficiency (-)
m.fs.pump.control_volume.properties_out[0].pressure.fix(30e5)          # pump outlet pressure (Pa)

# BW30-400 parameters, 7 degrees of freedom
m.fs.RO.A_comp.fix(9.63e-12)                                            # membrane water permeability coeff (m/Pa/s)
# m.fs.RO.A_comp.fix(4.2e-12)
m.fs.RO.B_comp.fix(5.58e-8)                                             # membrane salt permeability coeff (m/s)
# m.fs.RO.B_comp.fix(3.5e-8)
m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.9)                          # volumetric recovery (-) *

m.fs.RO.permeate.pressure[0].fix(101325)                               # permeate pressure (Pa)

# fix 5 module specficiations
# m.fs.RO.area.fix(50)                                             # membrane stage area (m^2)
# m.fs.RO.feed_side.velocity[0, 0].fix(0.15)                             # crossflow velocity (m/s) * try 0.25 m/s? 0.1 is on the low side
# m.fs.RO.feed_side.channel_height.fix(1e-3)                             # channel height in membrane stage (m), 0.75mm could also work
# m.fs.RO.feed_side.spacer_porosity.fix(0.97)                            # spacer porosity in membrane stage (-), try 0.85, .7ish on permeate

m.fs.separator2.split_fraction[0, "solids", "TDS"].fix(0.44)
m.fs.separator2.split_fraction[0, "solids", "H2O"].fix(1e-6)

# SW30-HRLE parameters, 7 degrees of freedom
# #m.fs.RO2.A_comp.fix(1.31e-12)                                            # membrane water permeability coeff (m/Pa/s)
m.fs.RO2.A_comp.fix(3.75e-12)
# #m.fs.RO2.B_comp.fix(8.61e-8)                                             # membrane salt permeability coeff (m/s)
m.fs.RO2.B_comp.fix(2.1e-7)   
m.fs.RO2.recovery_vol_phase[0, "Liq"].fix(0.5)                          # volumetric recovery (-) *
m.fs.RO2.permeate.pressure[0].fix(101325)                               # permeate pressure (Pa)
# m.fs.RO2.area.fix(50)                                             # membrane stage area (m^2)
# m.fs.RO2.feed_side.velocity[0, 0].fix(0.15)                             # crossflow velocity (m/s) * try 0.25 m/s? 0.1 is on the low side
# m.fs.RO2.feed_side.channel_height.fix(1e-3)                             # channel height in membrane stage (m), 0.75mm could also work
# m.fs.RO2.feed_side.spacer_porosity.fix(0.97)                            # spacer porosity in membrane stage (-), try 0.85, .7ish on permeate


#Prommis costing method adapted for this flowsheet
#Sum equipment
utilization = 1
TASC_TOC_factor = 1.207
CRF_factor = 0.0769
solids_mass = 1273.56 * 1.10231e-6 * 24 * 365 * 1 * utilization #ton/yr, from OLI, base 1273.56





default = 0
if default == 0:
    tic = 3.4
    mlcf = 0.043 #0.043 base
    mrf = 0.143 #0.143 base
    util = utilization
    m.fs.costing.high_pressure_pump.cost.fix(2.2) #$2023/W #base 2.2
    m.fs.costing.mixer.unit_cost.fix(1800) #$2023/L/s #base 1800
    m.fs.costing.reverse_osmosis.membrane_cost.fix(23) #$2023/m2 # base 23
    m.fs.costing.reverse_osmosis.high_pressure_membrane_cost.fix(25) #$2023/m2 # base 25
    m.fs.costing.reverse_osmosis.factor_membrane_replacement.fix(mrf)
    m.fs.costing.factor_capital_annualization.fix(CRF_factor)
    m.fs.costing.factor_maintenance_labor_chemical.fix(mlcf)
    m.fs.costing.factor_total_investment.fix(tic)
    m.fs.costing.utilization_factor.fix(util)
    m.fs.costing.electricity_cost.fix(eleccost)

else:
    tic = 2
    util = 0.9
    m.fs.costing.high_pressure_pump.cost.fix(1.908*cepci2023/cepci2018) #$2023/W
    m.fs.costing.mixer.unit_cost.fix(361*cepci2023/cepci2018) #$2023/L/s
    m.fs.costing.reverse_osmosis.membrane_cost.fix(30*cepci2023/cepci2018) #$2023/m2
    m.fs.costing.reverse_osmosis.high_pressure_membrane_cost.fix(30*cepci2023/cepci2018) #$2023/m2
    m.fs.costing.reverse_osmosis.factor_membrane_replacement.fix(0.1)
    m.fs.costing.factor_capital_annualization.fix(0.1)
    m.fs.costing.factor_maintenance_labor_chemical.fix(0.03)
    m.fs.costing.factor_total_investment.fix(tic)
    m.fs.costing.utilization_factor.fix(util)
    m.fs.costing.electricity_cost.fix(eleccost)


m.fs.costing.antiscalant_cost.fix(4.5e-6) #from Voutchkov, 0.5 - 2 mg antiscalant / L flow, $1.8-$4.4/kg in 2013 dollars, used averages for both, base 4.5e-6

print("DOF = ", degrees_of_freedom(m))

# initialize unit by unit
solver = get_solver()

# solve feed
solver.solve(m.fs.feed)

# initialize mixer
propagate_state(m.fs.s1)
m.fs.mixer.initialize()

#initialize separator
propagate_state(m.fs.s2)
m.fs.separator.initialize()
propagate_state(m.fs.s_s)

# initialize pump
propagate_state(m.fs.s3)
m.fs.pump.initialize()

# initialize RO
propagate_state(m.fs.s4)
m.fs.RO.initialize()

# propagate to product and disposal
propagate_state(m.fs.s5)
propagate_state(m.fs.s6)
m.fs.separator2.initialize()
propagate_state(m.fs.s_s2)

propagate_state(m.fs.s7)
propagate_state(m.fs.s7_1)
m.fs.RO2.initialize()

propagate_state(m.fs.s8)
propagate_state(m.fs.s9)
# initialize cost
m.fs.costing.initialize()

solver.solve(m.fs.brine)
# solve model
results = solver.solve(m) 
print(results)
# m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.9) 
# m.fs.RO.area.unfix()
# m.fs.RO2.recovery_vol_phase[0, "Liq"].fix(0.5)
# m.fs.RO2.area.unfix()
# solver = get_solver()
# results = solver.solve(m)





print("pump capital cost is ", value(m.fs.pump.costing.capital_cost)*tic)
print("RO capital cost is ", value(m.fs.RO.costing.capital_cost)*tic)
print("RO2 capital cost is ", value(m.fs.RO2.costing.capital_cost)*tic)
print("Mixer capital cost is ", value(m.fs.mixer.costing.capital_cost)*tic)
print("Membrane replacement cost is ", (value(m.fs.RO.costing.fixed_operating_cost)+value(m.fs.RO2.costing.fixed_operating_cost)))
print("Antiscalant cost is ", value(m.fs.costing.aggregate_flow_costs["antiscalant"]))
print("MLC cost is ", value(m.fs.costing.maintenance_labor_chemical_operating_cost))
orecov = (value(units.convert(m.fs.product.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr))+value(units.convert(m.fs.product2.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)))/value(units.convert(m.fs.feed.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr))*100

m.fs.costing.total_equip = m.fs.pump.costing.capital_cost + m.fs.mixer.costing.capital_cost + m.fs.RO.costing.capital_cost + m.fs.RO2.costing.capital_cost


#Calculate BEC

m.fs.costing.total_BEC = Expression(expr=m.fs.costing.total_equip * 1.58)

#Calculate ancillary costs
m.fs.costing.piping_MandL = Expression(expr=m.fs.costing.total_BEC * 0.2)
m.fs.costing.elec_MandL = Expression(expr=m.fs.costing.total_BEC * 0.2)
m.fs.costing.instrument = Expression(expr=m.fs.costing.total_BEC * 0.08)
m.fs.costing.plant_serv = Expression(expr=m.fs.costing.total_BEC * 0.1)

m.fs.costing.ancillary_cost = Expression(expr=(m.fs.costing.piping_MandL + m.fs.costing.elec_MandL + m.fs.costing.instrument + m.fs.costing.plant_serv))

#calculate building costs
m.fs.costing.proc_build = Expression(expr=m.fs.costing.total_BEC * 0.4)
m.fs.costing.aux_build = Expression(expr=m.fs.costing.total_BEC * 0.15)
m.fs.costing.sic_build = Expression(expr=m.fs.costing.total_BEC * 0.1)

m.fs.costing.building_cost = Expression(expr=(m.fs.costing.proc_build + m.fs.costing.aux_build + m.fs.costing.sic_build))

#calculate EPCM costs
m.fs.costing.eic = Expression(expr=m.fs.costing.total_BEC * 0.17)
m.fs.costing.fec = Expression(expr=m.fs.costing.total_BEC * 0.12)
m.fs.costing.pmcc = Expression(expr=m.fs.costing.total_BEC * 0.3)

m.fs.costing.epcm_cost = Expression(expr=(m.fs.costing.eic+m.fs.costing.fec+m.fs.costing.pmcc))

#Contingency costs
m.fs.costing.contingency_cost = Expression(expr=(0.15*m.fs.costing.total_BEC))

#Total installed costs
m.fs.costing.TIC_cost = Expression(expr=(m.fs.costing.ancillary_cost+m.fs.costing.building_cost+m.fs.costing.epcm_cost+m.fs.costing.contingency_cost))

#Total plant costs, total overnight cost
m.fs.costing.TPC_cost = Expression(expr=(m.fs.costing.total_BEC+m.fs.costing.TIC_cost))
m.fs.costing.TOC_cost = Expression(expr=(m.fs.costing.TPC_cost))

#TASC and annualized capital costs


m.fs.costing.TASC_cost = Expression(expr=(TASC_TOC_factor * m.fs.costing.TOC_cost))
m.fs.costing.prommis_annualized_capital_cost = Expression(expr=(CRF_factor*m.fs.costing.TASC_cost))

#Fixed operating costs

#Labor costs (no technical labor)

#Operating Labor
N_operator = 2
operator_cost = 24.81*1
t_shift = 8 #hours per shift
N_shifts = 3 #number of shifts per day
N_days = 365*utilization
labor_burden = 0.25

m.fs.costing.prommis_operating_labor_cost = Expression(expr=(1*N_operator*operator_cost*(1+labor_burden)*t_shift*eciratio*N_shifts*N_days*units.USD_2018))

#maintenance and material costs
m.fs.costing.MM_cost = Expression(expr=0.02*m.fs.costing.TPC_cost)

#QAQC costs
m.fs.costing.QAQC_cost = Expression(expr=0.1*m.fs.costing.prommis_operating_labor_cost)

#Admin labor costs
m.fs.costing.Admin_labor_cost = Expression(expr=0.2*m.fs.costing.prommis_operating_labor_cost)

#Patent costs are ignored (no sales yet)
#Property taxes

m.fs.costing.prop_tax_insurance_cost = Expression(expr=0.01*m.fs.costing.TPC_cost)

#WaterTAP costs are included here for fixed operating costs
#For this flowsheet, only membrane replacement is needed

m.fs.costing.WaterTAP_FOM_costs = Expression(expr=(m.fs.RO.costing.fixed_operating_cost + m.fs.RO2.costing.fixed_operating_cost))

#Summing relevant costs
m.fs.costing.prommis_fixed_operating_cost = Expression(expr=(m.fs.costing.prommis_operating_labor_cost+m.fs.costing.MM_cost+m.fs.costing.QAQC_cost+m.fs.costing.Admin_labor_cost+m.fs.costing.prop_tax_insurance_cost+m.fs.costing.WaterTAP_FOM_costs))

#Variable operating costs
#land costs are set to 0

#Per resource (waste disposal, antiscalant, electricity)
#Note that solid masses are taken from OLI (not calculated in WaterTAP)
m.fs.costing.liquid_waste_resource_cost = Expression(expr=(value(units.convert(m.fs.brine.properties[0].flow_vol_phase["Liq"],to_units=units.m ** 3 / units.hr))*1*264.172/42*24*365*1.5*1*utilization*units.USD_2018))
m.fs.costing.solid_waste_resource_cost = Expression(expr=solids_mass*1*units.USD_2018)
m.fs.costing.VOP_resource_cost = Expression(expr=(m.fs.costing.aggregate_flow_costs["antiscalant"]*utilization + m.fs.costing.aggregate_flow_costs["electricity"]*utilization+m.fs.costing.liquid_waste_resource_cost+m.fs.costing.solid_waste_resource_cost))

m.fs.costing.plant_overhead_cost = Expression(expr=(0.2*(m.fs.costing.prommis_fixed_operating_cost+m.fs.costing.VOP_resource_cost)))

m.fs.costing.prommis_variable_operating_cost = Expression(expr=(m.fs.costing.VOP_resource_cost+m.fs.costing.plant_overhead_cost))
#Annualized costs, prommis LCOW

m.fs.costing.prommis_annualized_cost = Expression(expr=(m.fs.costing.prommis_annualized_capital_cost+m.fs.costing.prommis_fixed_operating_cost+m.fs.costing.prommis_variable_operating_cost))

m.fs.costing.prommis_LCOW = Expression(expr=(m.fs.costing.prommis_annualized_cost/
                                             (value(units.convert(m.fs.product.properties[0].flow_vol+m.fs.product2.properties[0].flow_vol,
                                                                  to_units = units.m ** 3 / units.hr))*24*365*utilization)))

#cost scaling from the NETL QGESS https://www.netl.doe.gov/projects/files/QGESSCostEstMethodforNETLAssessmentsofPowerPlantPerformance_022621.pdf

pump_bec = value(m.fs.pump.costing.capital_cost) * (1 + 0.55)
mem_bec = (value(m.fs.RO.costing.capital_cost) + value(m.fs.RO2.costing.capital_cost)) * (1 + 0.55)
mixer_bec = value(m.fs.mixer.costing.capital_cost) * (1 + 0.55)

bec = pump_bec + mem_bec
epc_f = 0.2
proc_f = 0.4
proj_f = 0.2
tpc = (bec * (1+epc_f+proc_f))*(1+proj_f)

def toca(lab,mat,cons,waste,tpc):
    op6 = lab/2
    mat1 = mat/12
    cons1 = cons/12
    wast1 = waste/12
    start = op6+mat1+cons1+wast1+tpc*0.02
    cons2 = cons/6
    inv = cons2+0.005*tpc
    fin = 0.027*tpc
    other = 0.15*tpc
    toc = start+inv+fin+other+tpc
    return toc

def fcrf(life):
    etax = 0.2574 #effective tax rate assuming 6% state and 21% federal
    atwacc = 0.0473 #after tax working average cost of capital, calculated based on 55% debt, 45% equity, 5% nominal interest,
    #10% return on equity and the above effective tax rate
    fr = 1 #fraction that is depreciable, set to 1 based on QGESS document
    dperc20=[0.0375,0.07219,0.06677,0.06177,0.05713,0.05285,0.04888,0.04522,0.04462,0.04461,0.04462,0.04461,0.04462,0.04461,0.04462,0.04461,0.04462,0.04461,0.04462,0.04461,0.02231]
    dperc15=[0.05,0.0950,0.0855,0.0770,0.0693,0.0623,0.0590,0.0590,0.0591,0.0590,0.0591,0.0590,0.0591,0.0590,0.0591,0.0295]
    dperc10=[0.075,0.1388,0.1179,0.1002,0.0874,0.0874,0.0874,0.0874,0.0874,0.0874,0.0437]
    dperc5=[0.15,0.255,0.1785,0.1666,0.1666,0.0833]
    dperc3=[0.25,0.375,0.25,0.125]
    sumd=0
    if life >= 25:
        dperc=dperc20
    elif life >= 20:
        dperc=dperc15
    elif life >= 15:
        dperc=dperc10
    elif life >= 10:
        dperc = dperc5
    elif life >= 5:
        dperc = dperc3
    crf = (atwacc*(1+atwacc)**life)/((1+atwacc)**life-1)
    #depreciation calculation
    for i in range(len(dperc)):
        expon = i+1
        delem=dperc[i]/((1+atwacc)**expon)
        sumd=sumd+delem
        
    de = (crf*fr*sumd)
    
    
    fcr = (crf/(1-etax))-(etax*de)/(1-etax)
    return fcr


# toc = 

print("Liquid waste resource cost is", value(m.fs.costing.liquid_waste_resource_cost))


    
def display_solution_watertap(m):
    print("----------system metrics----------")
    print("Recovery: %.1f %%" % (orecov))
    print("Specific energy: %.2f kWh/m3" % value(m.fs.costing.specific_energy_consumption))
    print("Levelized cost of water: %.2f $/m3" % value(m.fs.costing.LCOW))
    print("Capital Cost: %.2f $" % value(m.fs.costing.total_capital_cost))
    print("Operating Cost: %.2f $" % value(m.fs.costing.total_operating_cost))
    print("Aggregate electricity cost: %.2f $" % (value(m.fs.costing.aggregate_flow_costs["electricity"])*util))
#     print("Aggregate Electricity Cost: %.2f $" % value(m.fs.costing.aggregate_electricity_cost))
    print("FOM: %.2f $" % value(m.fs.costing.aggregate_fixed_operating_cost))
    print("VOM: %.2f $" % value(m.fs.costing.aggregate_variable_operating_cost))

    print("\n----------inlet and outlets----------")
    print("Feed: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.feed.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Solids: %.2f kg/h, %.0f ppm" %
          (value( (2.163*10**3) *units.convert(m.fs.solids.properties[0].flow_vol_phase["Liq"],
                               to_units= units.m ** 3 / units.hr)),
           value(m.fs.solids.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Product 1: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.product.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.product.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Inlet to SWRO: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.RO2inlet.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.RO2inlet.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Product 2: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.product2.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.product2.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))
    print("Brine: %.2f m3/h, %.0f ppm" %
          (value(units.convert(m.fs.brine.properties[0].flow_vol_phase["Liq"],
                               to_units=units.m ** 3 / units.hr)),
           value(m.fs.brine.properties[0].mass_frac_phase_comp["Liq", "TDS"]) * 1e6))

    print("\n----------decision variables----------")
    print("Operating pressure: %.1f bar" %
          (value(units.convert(m.fs.pump.control_volume.properties_out[0].pressure,
                               to_units=units.bar))))
    print("Membrane area 1: %.1f m2" % value(m.fs.RO.area))
    print("Membrane area 2: %.1f m2" % value(m.fs.RO2.area))
#     print("Inlet crossflow velocity: %.1f cm/s" %
#           (value(units.convert(m.fs.RO.feed_side.velocity[0, 0],
#                                to_units=units.cm / units.s))))

    print("\n----------system variables----------")
    print("Pump power: %.1f kW" %
          (value(units.convert(m.fs.pump.work_mechanical[0], to_units=units.kW))))
    print("Average water flux: %.1f L/(m2-h)" %
          value(units.convert(m.fs.RO.flux_mass_phase_comp_avg[0, "Liq", "H2O"]
                              / (1000 * units.kg / units.m ** 3),
                              to_units=units.mm / units.hr)))
    print("Average water flux 2: %.1f L/(m2-h)" %
          value(units.convert(m.fs.RO2.flux_mass_phase_comp_avg[0, "Liq", "H2O"]
                              / (1000 * units.kg / units.m ** 3),
                              to_units=units.mm / units.hr)))
#     print("Pressure drop: %.1f bar" %
#           (-value(units.convert(m.fs.RO.deltaP[0],to_units=units.bar))))
    print("Maximum interfacial salinity: %.0f ppm" %
          (value(m.fs.RO.feed_side.properties_interface[0, 1].mass_frac_phase_comp["Liq", "TDS"])*1e6))
    print("Maximum interfacial salinity: %.0f ppm" %
          (value(m.fs.RO2.feed_side.properties_interface[0, 1].mass_frac_phase_comp["Liq", "TDS"])*1e6))

display_solution_watertap(m)

outarray = []
col2 = []
col2.append("LCOW ($/m3)")
col2.append(value(m.fs.costing.LCOW))
outarray.append(col2)
col2 = []
col2.append("Total Capital Cost $")
col2.append(value(m.fs.costing.total_capital_cost))
outarray.append(col2)
col2 = []
col2.append("Pump Cost $")
col2.append(value(m.fs.pump.costing.capital_cost)*tic)
outarray.append(col2)
col2 = []
col2.append("BWRO Cost $")
col2.append(value(m.fs.RO.costing.capital_cost)*tic)
outarray.append(col2)
col2 = []
col2.append("SWRO Cost $")
col2.append(value(m.fs.RO2.costing.capital_cost)*tic)
outarray.append(col2)
col2 = []
col2.append("Tank Cost $")
col2.append(value(m.fs.mixer.costing.capital_cost)*tic)
outarray.append(col2)
col2 = []
col2.append("Total Operating Cost $/yr")
col2.append(value(m.fs.costing.total_operating_cost))
outarray.append(col2)
col2 = []
col2.append("Operating Labor Cost $/yr")
col2.append((value(m.fs.costing.prommis_operating_labor_cost)))
outarray.append(col2)
col2 = []
col2.append("Antiscalant Cost $/yr")
col2.append(value(m.fs.costing.aggregate_flow_costs["antiscalant"])*util)
outarray.append(col2)
col2 = []
col2.append("MLC Cost $/yr")
col2.append(value(m.fs.costing.maintenance_labor_chemical_operating_cost))
outarray.append(col2)
col2 = []
col2.append("Elec Cost $/yr")
col2.append(value(m.fs.costing.aggregate_flow_costs["electricity"])*util)
outarray.append(col2)

with open('watertaplandfill.csv','w',newline='') as out:
    write=csv.writer(out)
    for x in range(len(outarray)):
        write.writerow(outarray[x])
        
        
outarray = []
col2 = []
col2.append("Prommis LCOW ($/m3)")
col2.append(value(m.fs.costing.prommis_LCOW))
outarray.append(col2)
col2 = []
col2.append("Total As-Spent Capital Cost $")
col2.append(value(m.fs.costing.TASC_cost))
outarray.append(col2)
col2 = []
col2.append("Total Fixed Operating Cost $/yr")
col2.append(value(m.fs.costing.prommis_fixed_operating_cost))
outarray.append(col2)
col2 = []
col2.append("Total Operating Labor $/yr")
col2.append((value(m.fs.costing.prommis_operating_labor_cost)))
outarray.append(col2)
col2 = []
col2.append("Total Variable Operating Cost $/yr")
col2.append(value(m.fs.costing.prommis_variable_operating_cost))
outarray.append(col2)
col2 = []
col2.append("Liquid Waste Disposal Cost $/yr")
col2.append(value(m.fs.costing.liquid_waste_resource_cost))
outarray.append(col2)
col2 = []
col2.append("Solids Waste Disposal Cost $/yr")
col2.append(value(m.fs.costing.solid_waste_resource_cost))
outarray.append(col2)
col2 = []
col2.append("Plant Overhead Cost $/yr")
col2.append(value(m.fs.costing.plant_overhead_cost))
outarray.append(col2)

with open('watertapprommislandfill.csv','w',newline='') as out:
    write=csv.writer(out)
    for x in range(len(outarray)):
        write.writerow(outarray[x])
        
        



2024-06-10 15:29:56 [WARNING] idaes.core.base.property_meta: DEPRECATED: The property name boiling_point_elevation_phase in
property metadata is not a recognized standard property name defined
in this PropertySet. Please refer to IDAES standard names in the IDAES
documentation. You can use the define_custom_properties() rather than
the add_properties() method to define metadata for this property. You
can also use a different property set by calling the
define_property_set() method.  (deprecated in 2.0.0, will be removed
in (or after) 3.0.0)
(called from e:\anaconda3\envs\watertap\lib\site-packages\watertap\property_models\seawater_prop_pack.py:733)
DOF =  19
DOF =  0
2024-06-10 15:29:57 [INFO] idaes.init.fs.mixer.mixed_state: fs.mixer.mixed_state State Released.
2024-06-10 15:29:57 [INFO] idaes.init.fs.mixer: Initialization Complete: optimal - Optimal Solution Found
2024-06-10 15:29:57 [INFO] idaes.init.fs.mixer.feed_state: fs.mixer.feed_state State Released.
2024-06-10 15:29:57 [INFO]